# 이수안컴퓨터연구소 머신러닝 #6
#### Naive Bayes Classification
1. 가우시안 나이브 베이즈 분류기
2. 베르누이 나이브 베이즈 분류기
3. 다항 나이브 베이즈 분류기
#### K-Nearest Neighbor Resressor
1. Boston
2. California

In [1]:
import numpy as np
import pandas as pd

from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.datasets import fetch_covtype, fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn import metrics

In [2]:
prior = [0.45, 0.3, 0.15, 0.1]
likelihood = [[0.3, 0.3, 0.4], [0.7, 0.2, 0.1], [0.15, 0.5, 0.35], [0.6, 0.2, 0.2]]

idx = 0
for c, xs in zip(prior, likelihood):
    result = 1.

    for x in xs:
        result *= x
    result *= c

    idx += 1
    print(f'{idx} 번째 클래스의 가능성: {result}')

1 번째 클래스의 가능성: 0.0162
2 번째 클래스의 가능성: 0.0042
3 번째 클래스의 가능성: 0.0039375
4 번째 클래스의 가능성: 0.0024000000000000002


### 산림 토양 데이터

In [3]:
covtype = fetch_covtype()
print(covtype.DESCR)

.. _covtype_dataset:

Forest covertypes
-----------------

The samples in this dataset correspond to 30×30m patches of forest in the US,
collected for the task of predicting each patch's cover type,
i.e. the dominant species of tree.
There are seven covertypes, making this a multiclass classification problem.
Each sample has 54 features, described on the
`dataset's homepage <https://archive.ics.uci.edu/ml/datasets/Covertype>`__.
Some of the features are boolean indicators,
while others are discrete or continuous measurements.

**Data Set Characteristics:**

    =================   ============
    Classes                        7
    Samples total             581012
    Dimensionality                54
    Features                     int
    =================   ============

:func:`sklearn.datasets.fetch_covtype` will load the covertype dataset;
it returns a dictionary-like 'Bunch' object
with the feature matrix in the ``data`` member
and the target values in ``target``. If optional a

In [4]:
pd.DataFrame(covtype.data, columns=covtype.feature_names).head(2)

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type_30,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39
0,2596.0,51.0,3.0,258.0,0.0,510.0,221.0,232.0,148.0,6279.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2590.0,56.0,2.0,212.0,-6.0,390.0,220.0,235.0,151.0,6225.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
covtype_X, covtype_y = fetch_covtype(return_X_y=True)

In [13]:
covtype_X_trian, covtype_X_test, covtype_y_train, covtype_y_test = train_test_split(covtype_X, covtype_y, test_size=0.2)

In [14]:
print(f'전체 데이터 크기: {covtype_X.shape}')
print(f'학습 데이터 크기: {covtype_X_trian.shape}')
print(f'평가 데이터 크기: {covtype_X_test.shape}')

전체 데이터 크기: (581012, 54)
학습 데이터 크기: (464809, 54)
평가 데이터 크기: (116203, 54)


In [15]:
pd.DataFrame(covtype.data, columns=covtype.feature_names).describe()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type_30,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39
count,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,...,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000
mean,2959.365301,155.656807,14.103704,269.428217,46.418855,2350.146611,212.146049,223.318716,142.528263,1980.291226,...,0.044175,0.090392,0.077716,0.002773,0.003255,0.000205,0.000513,0.026803,0.023762,0.015060
std,279.984734,111.913721,7.488242,212.549356,58.295232,1559.254870,26.769889,19.768697,38.274529,1324.195210,...,0.205483,0.286743,0.267725,0.052584,0.056957,0.014310,0.022641,0.161508,0.152307,0.121791
min,1859.000000,0.000000,0.000000,0.000000,-173.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2809.000000,58.000000,9.000000,108.000000,7.000000,1106.000000,198.000000,213.000000,119.000000,1024.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2996.000000,127.000000,13.000000,218.000000,30.000000,1997.000000,218.000000,226.000000,143.000000,1710.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3163.000000,260.000000,18.000000,384.000000,69.000000,3328.000000,231.000000,237.000000,168.000000,2550.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,3858.000000,360.000000,66.000000,1397.000000,601.000000,7117.000000,254.000000,254.000000,254.000000,7173.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [16]:
scaler = StandardScaler()
covtype_X_train_scale = scaler.fit_transform(covtype_X_trian)
covtype_X_test_scale = scaler.transform(covtype_X_test)

In [17]:
covtype_train_df = pd.DataFrame(covtype_X_train_scale, columns=covtype.feature_names)
covtype_train_df.describe()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type_30,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39
count,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05,...,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05
mean,-1.573773e-16,4.032649e-17,-5.319795e-18,3.439523e-17,-8.621737e-18,-1.712118e-17,3.399777e-16,-4.573495e-16,-2.961200e-16,6.175854e-18,...,1.281031e-17,9.386076e-18,2.277728e-18,5.140940e-17,-4.693038e-17,1.574537e-17,-1.253515e-18,-5.491007e-17,4.280295e-17,-2.935059e-17
std,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,...,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00
min,-3.930882e+00,-1.390779e+00,-1.883993e+00,-1.268265e+00,-3.763908e+00,-1.506146e+00,-7.931331e+00,-1.129364e+01,-3.723888e+00,-1.496382e+00,...,-2.154733e-01,-3.153859e-01,-2.907319e-01,-5.242560e-02,-5.630682e-02,-1.414646e-02,-2.210455e-02,-1.665684e-01,-1.559866e-01,-1.238866e-01
25%,-5.372638e-01,-8.724449e-01,-6.816371e-01,-7.599306e-01,-6.768010e-01,-7.988309e-01,-5.292310e-01,-5.217279e-01,-6.144784e-01,-7.222357e-01,...,-2.154733e-01,-3.153859e-01,-2.907319e-01,-5.242560e-02,-5.630682e-02,-1.414646e-02,-2.210455e-02,-1.665684e-01,-1.559866e-01,-1.238866e-01
50%,1.307432e-01,-2.558062e-01,-1.472567e-01,-2.421831e-01,-2.823373e-01,-2.268226e-01,2.184559e-01,1.357129e-01,1.262944e-02,-2.036180e-01,...,-2.154733e-01,-3.153859e-01,-2.907319e-01,-5.242560e-02,-5.630682e-02,-1.414646e-02,-2.210455e-02,-1.665684e-01,-1.559866e-01,-1.238866e-01
75%,7.273055e-01,9.327872e-01,5.207187e-01,5.391451e-01,3.865359e-01,6.267010e-01,7.044523e-01,6.920089e-01,6.658668e-01,4.306682e-01,...,-2.154733e-01,-3.153859e-01,-2.907319e-01,-5.242560e-02,-5.630682e-02,-1.414646e-02,-2.210455e-02,-1.665684e-01,-1.559866e-01,-1.238866e-01
max,3.210005e+00,1.826466e+00,6.933283e+00,5.307130e+00,9.510653e+00,3.044911e+00,1.564292e+00,1.551739e+00,2.913003e+00,3.926424e+00,...,4.640946e+00,3.170719e+00,3.439595e+00,1.907465e+01,1.775984e+01,7.068908e+01,4.523955e+01,6.003539e+00,6.410807e+00,8.071900e+00


In [19]:
newsgroup = fetch_20newsgroups()
print(newsgroup.DESCR)

.. _20newsgroups_dataset:

The 20 newsgroups text dataset
------------------------------

The 20 newsgroups dataset comprises around 18000 newsgroups posts on
20 topics split in two subsets: one for training (or development)
and the other one for testing (or for performance evaluation). The split
between the train and test set is based upon a messages posted before
and after a specific date.

This module contains two loaders. The first one,
:func:`sklearn.datasets.fetch_20newsgroups`,
returns a list of the raw texts that can be fed to text feature
extractors such as :class:`~sklearn.feature_extraction.text.CountVectorizer`
with custom parameters so as to extract feature vectors.
The second one, :func:`sklearn.datasets.fetch_20newsgroups_vectorized`,
returns ready-to-use features, i.e., it is not necessary to use a feature
extractor.

**Data Set Characteristics:**

    =================   ==========
    Classes                     20
    Samples total            18846
    Dimensionality